<a href="https://colab.research.google.com/github/wayne0git/ml_cv_basics/blob/master/image_synthesis/stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Examples
#### Adapted from: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb

## Environment Preparation

### Install Library

In [ ]:
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

### Import Library

In [2]:
import os
import cv2
import numpy as np

from PIL import Image, ImageDraw
from base64 import b64encode
from tqdm.auto import tqdm

In [3]:
from IPython.display import HTML
from google.colab import output

In [4]:
import torch
from torch import autocast
from torch.nn import functional as F

from diffusers import StableDiffusionPipeline, AutoencoderKL
from diffusers import UNet2DConditionModel, PNDMScheduler, LMSDiscreteScheduler
from diffusers.schedulers.scheduling_ddim import DDIMScheduler

from transformers import CLIPTextModel, CLIPTokenizer

In [5]:
from huggingface_hub import notebook_login

### Constant

In [6]:
device = 'cuda'

### Configure Colab

In [7]:
output.enable_custom_widget_manager()

### Login huggingface_hub

In [8]:
notebook_login()

Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


### Utility Function

In [9]:
def image_grid(imgs, rows, cols):
  assert len(imgs) == rows * cols

  w, h = imgs[0].size
  grid = Image.new('RGB', size=(cols * w, rows * h))
  grid_w, grid_h = grid.size
   
  for i, img in enumerate(imgs):
    grid.paste(img, box=(i % cols * w, i // cols * h))

  return grid

In [10]:
def imgs_to_video(imgs, video_name='video.mp4', fps=15):
  # Source: https://stackoverflow.com/questions/52414148/turn-pil-images-into-video-on-linux
  video_dims = (imgs[0].width, imgs[0].height)
  fourcc = cv2.VideoWriter_fourcc(*'DIVX')    
  video = cv2.VideoWriter(video_name, fourcc, fps, video_dims)

  for img in imgs:
    tmp_img = img.copy()
    video.write(cv2.cvtColor(np.array(tmp_img), cv2.COLOR_RGB2BGR))

  video.release()

In [11]:
def display_video(file_path, width=512):
  # Compress video
  compressed_vid_path = 'comp_' + file_path
  if os.path.exists(compressed_vid_path):
    os.remove(compressed_vid_path)

  os.system(f'ffmpeg -i {file_path} -vcodec libx264 {compressed_vid_path}')

  # Base64 encode
  mp4 = open(compressed_vid_path, 'rb').read()
  data_url = 'data:simul2/mp4;base64,' + b64encode(mp4).decode()

  return HTML("""
    <video width={} controls>
          <source src="{}" type="video/mp4">
    </video>
    """.format(width, data_url))

## Run stable diffusion (Simple)

In [ ]:
# Initalize stable diffusion model
# Should login & visit https://huggingface.co/CompVis/stable-diffusion-v1-4,
# Read, accept and acknowledge the LICENSE
pipe = StableDiffusionPipeline.from_pretrained('CompVis/stable-diffusion-v1-4', revision='fp16',
                          torch_dtype=torch.float16, use_auth_token=True)
pipe = pipe.to(device)

In [ ]:
# Parameter
width, height = 512, 512
n_step = 50
prompts = ['Cute shiba inu dog'] * 3

# Run stable diffusion
with autocast(device):
  result = pipe(prompts, width=width, height=height, num_inference_steps=n_step)
  images = result['sample']

# Show result
image_grid(images, rows=1, cols=3)

## Run stable diffusion (Pipeline)

### Load Model

In [ ]:
# 1. Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained('CompVis/stable-diffusion-v1-4', subfolder='vae', use_auth_token=True)
vae = vae.to(device)

In [ ]:
# 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = CLIPTextModel.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = text_encoder.to(device)

In [ ]:
# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained('CompVis/stable-diffusion-v1-4', subfolder='unet', use_auth_token=True)
unet = unet.to(device)

In [46]:
# 4. Create a scheduler for inference
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule='scaled_linear', num_train_timesteps=1000)

### Function block of stable diffusion

In [18]:
def get_text_embeds(prompt):
  # Tokenize text and get embeddings
  text_input = tokenizer(prompt, padding='max_length', max_length=tokenizer.model_max_length,
               truncation=True, return_tensors='pt')
  with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(device))[0]

  # Do the same for unconditional embeddings
  uncond_input = tokenizer([''] * len(prompt), padding='max_length', max_length=tokenizer.model_max_length,
                return_tensors='pt')
  with torch.no_grad():
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(device))[0]

  # Cat for final embeddings (2, 77, 768)
  text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

  return text_embeddings

In [ ]:
# Test get_text_embeds
test_embeds = get_text_embeds(['Cute shiba inu dog'])
print(test_embeds.shape)

In [47]:
def produce_latents(text_embeddings, height=512, width=512, 
           num_inference_steps=50, guidance_scale=7.5, latents=None,
           return_all_latents=False, start_step=0):
  # Initialize latent (1, 4, 64, 64)
  if latents is None:
    latents = torch.randn((text_embeddings.shape[0] // 2, unet.in_channels, height // 8, width // 8))
  latents = latents.to(device)

  scheduler.set_timesteps(num_inference_steps)
  latents = latents * scheduler.sigmas[0]

  # Diffusion process given start step
  if start_step > 0:
    start_timestep = scheduler.timesteps[start_step]
    start_timesteps = start_timestep.repeat(latents.shape[0]).long()

    noise = torch.randn_like(latents)
    latents = scheduler.add_noise(latents, noise, start_timesteps)

  # Reverse the diffusion process iteratively
  latent_hist = [latents]

  with autocast('cuda'):
    for i, t in tqdm(enumerate(scheduler.timesteps[start_step:])):
      # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
      latent_model_input = torch.cat([latents] * 2)
      sigma = scheduler.sigmas[i]
      latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)

      # predict the noise residual
      with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)['sample']

      # perform guidance
      noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
      noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

      # compute the previous noisy sample x_t -> x_t-1
      latents = scheduler.step(noise_pred, i, latents)['prev_sample']
      latent_hist.append(latents)
  
  if return_all_latents:
    all_latents = torch.cat(latent_hist, dim=0)
    return all_latents      
  else:
    return latents

In [ ]:
# Test produce_latents
test_latents = produce_latents(test_embeds)
print(test_latents.shape)

In [19]:
def decode_img_latents(latents):
  # Normalization
  latents = latents / 0.18215

  # Decode
  with torch.no_grad():
    imgs = vae.decode(latents)

  # Tensor => Images
  imgs = (imgs / 2 + 0.5).clamp(0, 1)
  imgs = imgs.detach().cpu().permute(0, 2, 3, 1).numpy()
  imgs = (imgs * 255).round().astype('uint8')
  pil_images = [Image.fromarray(image) for image in imgs]

  return pil_images

In [ ]:
# Test decode_img_latents
imgs = decode_img_latents(test_latents)
imgs[0]

### Stable Diffusion Pipeline

In [24]:
def prompt_to_img(prompts, height=512, width=512, num_inference_steps=50,
          guidance_scale=7.5, latents=None,
          return_all_latents=False, batch_size=2, start_step=0):
  # Prompts -> text embeds
  text_embeds = get_text_embeds(prompts)

  # Text embeds -> img latents
  latents = produce_latents(text_embeds, height=height, width=width, latents=latents,
                 num_inference_steps=num_inference_steps, guidance_scale=guidance_scale,
                 return_all_latents=return_all_latents, start_step=start_step)
  
  # Img latents -> imgs
  all_imgs = []
  for i in tqdm(range(0, len(latents), batch_size)):
    imgs = decode_img_latents(latents[i:i+batch_size])
    all_imgs.extend(imgs)

  return all_imgs

#### Image

In [ ]:
prompt_to_img(['Cute shiba inu dog'], 512, 512, 50)[0]

#### Video

In [ ]:
video_frames = prompt_to_img(['Cute shiba inu dog'], num_inference_steps=40, return_all_latents=True)
imgs_to_video(video_frames, 'test.mp4')
display_video('test.mp4')

#### Similar Img

In [26]:
def perturb_latents(latents, scale=0.1):
  noise = torch.randn_like(latents)
  new_latents = (1 - scale) * latents + scale * noise

  return (new_latents - new_latents.mean()) / new_latents.std()

In [ ]:
# Create image from random noise
latents = torch.randn((1, unet.in_channels, 64, 64))
prompt_to_img(['Cute shiba inu dog'], 512, 512, 50, latents=latents)[0]

In [ ]:
# Create image from perturbed random noise
new_latents = perturb_latents(latents, 0.1)
prompt_to_img(['Cute shiba inu dog'], 512, 512, 50, latents=new_latents)[0]

#### Img-to-Img

In [30]:
def encode_img_latents(imgs):
  img_arr = np.stack([np.array(img) for img in imgs], axis=0)
  img_arr = img_arr / 255.0
  img_arr = torch.from_numpy(img_arr).float().permute(0, 3, 1, 2)
  img_arr = 2 * (img_arr - 0.5)

  latent_dists = vae.encode(img_arr.to(device))
  latent_samples = latent_dists.sample()
  latent_samples *= 0.18215

  return latent_samples

In [ ]:
# Create reference image
img = prompt_to_img(['Cute shiba inu dog'], 512, 512, num_inference_steps=30)[0]
img

In [33]:
# Encode reference image
img_latents = encode_img_latents([img])

In [ ]:
# Create new image based on reference image (May need different scheduler)
prompt_to_img(['Cute shiba inu dog under star sky'], 512, 512, num_inference_steps=30, latents=img_latents,start_step=20)[0]